d-sandbox
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 400px">
</div>

# Query Optimization
1. Logical optimizations
1. Predicate pushdown
1. No predicate pushdown

##### Methods
- DataFrame (<a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=dataframe#pyspark.sql.DataFrame" target="_blank">Python</a>/<a href="http://spark.apache.org/docs/latest/api/scala/org/apache/spark/sql/Dataset.html" target="_blank">Scala</a>): `explain`

In [0]:
%run ./Includes/Classroom-Setup

Datasets mounted and student environment set up

In [0]:
df = spark.read.parquet(eventsPath)
display(df.limit(5))

device,ecommerce,event_name,event_previous_timestamp,event_timestamp,geo,items,traffic_source,user_first_touch_timestamp,user_id
macOS,"List(null, null, null)",warranty,1593878899217692,1593878946592107,"List(Montrose, MI)",List(),google,1593878899217692,UA000000107379500
Windows,"List(null, null, null)",press,1593876662175340,1593877011756535,"List(Northampton, MA)",List(),google,1593876662175340,UA000000107359357
macOS,"List(null, null, null)",add_item,1593878792892652,1593878815459100,"List(Salinas, CA)","List(List(null, M_STAN_T, Standard Twin Mattress, 595.0, 595.0, 1))",youtube,1593878455472030,UA000000107375547
iOS,"List(null, null, null)",mattresses,1593878178791663,1593878809276923,"List(Everett, MA)",List(),facebook,1593877903116176,UA000000107370581
Windows,"List(null, null, null)",mattresses,null,1593878628143633,"List(Cottage Grove, MN)",List(),google,1593878628143633,UA000000107377108


### ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Logical Optimization

#### `explain(..)`

Prints the plans (logical and physical), optionally formatted by a given explain mode

In [0]:
from pyspark.sql.functions import col

limitEventsDF = (df
  .filter(col("event_name") != "reviews")
  .filter(col("event_name") != "checkout")
  .filter(col("event_name") != "register")
  .filter(col("event_name") != "email_coupon")
  .filter(col("event_name") != "cc_info")
  .filter(col("event_name") != "delivery")
  .filter(col("event_name") != "shipping_info")
  .filter(col("event_name") != "press")
)

limitEventsDF.count()

limitEventsDF.explain(True)

== Parsed Logical Plan ==
'Filter NOT ('event_name = press)
+- Filter NOT (event_name#1206 = shipping_info)
 +- Filter NOT (event_name#1206 = delivery)
 +- Filter NOT (event_name#1206 = cc_info)
 +- Filter NOT (event_name#1206 = email_coupon)
 +- Filter NOT (event_name#1206 = register)
 +- Filter NOT (event_name#1206 = checkout)
 +- Filter NOT (event_name#1206 = reviews)
 +- Relation[device#1204,ecommerce#1205,event_name#1206,event_previous_timestamp#1207L,event_timestamp#1208L,geo#1209,items#1210,traffic_source#1211,user_first_touch_timestamp#1212L,user_id#1213] parquet

== Analyzed Logical Plan ==
device: string, ecommerce: struct<purchase_revenue_in_usd:double,total_item_quantity:bigint,unique_items:bigint>, event_name: string, event_previous_timestamp: bigint, event_timestamp: bigint, geo: struct<city:string,state:string>, items: array<struct<coupon:string,item_id:string,item_name:string,item_revenue_in_usd:double,price_in_usd:double,quantity:bigint>>, traffic_source: string, user_first_touch_timestamp: bigint, user_id: string
Filter NOT (event_name#1206 = press)
+- Filter NOT (event_name#1206 = shipping_info)
 +- Filter NOT (event_name#1206 = delivery)
 +- Filter NOT (event_name#1206 = cc_info)
 +- Filter NOT (event_name#1206 = email_coupon)
 +- Filter NOT (event_name#1206 = register)
 +- Filter NOT (event_name#1206 = checkout)
 +- Filter NOT (event_name#1206 = reviews)
 +- Relation[device#1204,ecommerce#1205,event_name#1206,event_previous_timestamp#1207L,event_timestamp#1208L,geo#1209,items#1210,traffic_source#1211,user_first_touch_timestamp#1212L,user_id#1213] parquet

== Optimized Logical Plan ==
Filter ((((((((isnotnull(event_name#1206) AND NOT (event_name#1206 = reviews)) AND NOT (event_name#1206 = checkout)) AND NOT (event_name#1206 = register)) AND NOT (event_name#1206 = email_coupon)) AND NOT (event_name#1206 = cc_info)) AND NOT (event_name#1206 = delivery)) AND NOT (event_name#1206 = shipping_info)) AND NOT (event_name#1206 = press))
+- Relation[device#1204,ecommerce#1205,event_name#1206,event_previous_timestamp#1207L,event_timestamp#1208L,geo#1209,items#1210,traffic_source#1211,user_first_touch_timestamp#1212L,user_id#1213] parquet

== Physical Plan ==
*(1) Filter ((((((((isnotnull(event_name#1206) AND NOT (event_name#1206 = reviews)) AND NOT (event_name#1206 = checkout)) AND NOT (event_name#1206 = register)) AND NOT (event_name#1206 = email_coupon)) AND NOT (event_name#1206 = cc_info)) AND NOT (event_name#1206 = delivery)) AND NOT (event_name#1206 = shipping_info)) AND NOT (event_name#1206 = press))
+- *(1) ColumnarToRow
 +- FileScan parquet [device#1204,ecommerce#1205,event_name#1206,event_previous_timestamp#1207L,event_timestamp#1208L,geo#1209,items#1210,traffic_source#1211,user_first_touch_timestamp#1212L,user_id#1213] Batched: true, DataFilters: [isnotnull(event_name#1206), NOT (event_name#1206 = reviews), NOT (event_name#1206 = checkout), N..., Format: Parquet, Location: InMemoryFileIndex[dbfs:/mnt/training/ecommerce/events/events.parquet], PartitionFilters: [], PushedFilters: [IsNotNull(event_name), Not(EqualTo(event_name,reviews)), Not(EqualTo(event_name,checkout)), Not(..., ReadSchema: struct<device:string,ecommerce:struct<purchase_revenue_in_usd:double,total_item_quantity:bigint,u...

In [0]:
betterDF = (df.filter( 
  (col("event_name").isNotNull()) &
  (col("event_name") != "reviews") & 
  (col("event_name") != "checkout") & 
  (col("event_name") != "register") & 
  (col("event_name") != "email_coupon") & 
  (col("event_name") != "cc_info") & 
  (col("event_name") != "delivery") & 
  (col("event_name") != "shipping_info") & 
  (col("event_name") != "press")
))

betterDF.count()

betterDF.explain(True)

== Parsed Logical Plan ==
'Filter ((((((((isnotnull('event_name) AND NOT ('event_name = reviews)) AND NOT ('event_name = checkout)) AND NOT ('event_name = register)) AND NOT ('event_name = email_coupon)) AND NOT ('event_name = cc_info)) AND NOT ('event_name = delivery)) AND NOT ('event_name = shipping_info)) AND NOT ('event_name = press))
+- Relation[device#1204,ecommerce#1205,event_name#1206,event_previous_timestamp#1207L,event_timestamp#1208L,geo#1209,items#1210,traffic_source#1211,user_first_touch_timestamp#1212L,user_id#1213] parquet

== Analyzed Logical Plan ==
device: string, ecommerce: struct<purchase_revenue_in_usd:double,total_item_quantity:bigint,unique_items:bigint>, event_name: string, event_previous_timestamp: bigint, event_timestamp: bigint, geo: struct<city:string,state:string>, items: array<struct<coupon:string,item_id:string,item_name:string,item_revenue_in_usd:double,price_in_usd:double,quantity:bigint>>, traffic_source: string, user_first_touch_timestamp: bigint, user_id: string
Filter ((((((((isnotnull(event_name#1206) AND NOT (event_name#1206 = reviews)) AND NOT (event_name#1206 = checkout)) AND NOT (event_name#1206 = register)) AND NOT (event_name#1206 = email_coupon)) AND NOT (event_name#1206 = cc_info)) AND NOT (event_name#1206 = delivery)) AND NOT (event_name#1206 = shipping_info)) AND NOT (event_name#1206 = press))
+- Relation[device#1204,ecommerce#1205,event_name#1206,event_previous_timestamp#1207L,event_timestamp#1208L,geo#1209,items#1210,traffic_source#1211,user_first_touch_timestamp#1212L,user_id#1213] parquet

== Optimized Logical Plan ==
Filter ((((((((isnotnull(event_name#1206) AND NOT (event_name#1206 = reviews)) AND NOT (event_name#1206 = checkout)) AND NOT (event_name#1206 = register)) AND NOT (event_name#1206 = email_coupon)) AND NOT (event_name#1206 = cc_info)) AND NOT (event_name#1206 = delivery)) AND NOT (event_name#1206 = shipping_info)) AND NOT (event_name#1206 = press))
+- Relation[device#1204,ecommerce#1205,event_name#1206,event_previous_timestamp#1207L,event_timestamp#1208L,geo#1209,items#1210,traffic_source#1211,user_first_touch_timestamp#1212L,user_id#1213] parquet

== Physical Plan ==
*(1) Filter ((((((((isnotnull(event_name#1206) AND NOT (event_name#1206 = reviews)) AND NOT (event_name#1206 = checkout)) AND NOT (event_name#1206 = register)) AND NOT (event_name#1206 = email_coupon)) AND NOT (event_name#1206 = cc_info)) AND NOT (event_name#1206 = delivery)) AND NOT (event_name#1206 = shipping_info)) AND NOT (event_name#1206 = press))
+- *(1) ColumnarToRow
 +- FileScan parquet [device#1204,ecommerce#1205,event_name#1206,event_previous_timestamp#1207L,event_timestamp#1208L,geo#1209,items#1210,traffic_source#1211,user_first_touch_timestamp#1212L,user_id#1213] Batched: true, DataFilters: [isnotnull(event_name#1206), NOT (event_name#1206 = reviews), NOT (event_name#1206 = checkout), N..., Format: Parquet, Location: InMemoryFileIndex[dbfs:/mnt/training/ecommerce/events/events.parquet], PartitionFilters: [], PushedFilters: [IsNotNull(event_name), Not(EqualTo(event_name,reviews)), Not(EqualTo(event_name,checkout)), Not(..., ReadSchema: struct<device:string,ecommerce:struct<purchase_revenue_in_usd:double,total_item_quantity:bigint,u...

In [0]:
stupidDF = (df
  .filter(col("event_name") != "finalize")
  .filter(col("event_name") != "finalize")
  .filter(col("event_name") != "finalize")
  .filter(col("event_name") != "finalize")
  .filter(col("event_name") != "finalize")
)

stupidDF.explain(True)

== Parsed Logical Plan ==
'Filter NOT ('event_name = finalize)
+- Filter NOT (event_name#1206 = finalize)
 +- Filter NOT (event_name#1206 = finalize)
 +- Filter NOT (event_name#1206 = finalize)
 +- Filter NOT (event_name#1206 = finalize)
 +- Relation[device#1204,ecommerce#1205,event_name#1206,event_previous_timestamp#1207L,event_timestamp#1208L,geo#1209,items#1210,traffic_source#1211,user_first_touch_timestamp#1212L,user_id#1213] parquet

== Analyzed Logical Plan ==
device: string, ecommerce: struct<purchase_revenue_in_usd:double,total_item_quantity:bigint,unique_items:bigint>, event_name: string, event_previous_timestamp: bigint, event_timestamp: bigint, geo: struct<city:string,state:string>, items: array<struct<coupon:string,item_id:string,item_name:string,item_revenue_in_usd:double,price_in_usd:double,quantity:bigint>>, traffic_source: string, user_first_touch_timestamp: bigint, user_id: string
Filter NOT (event_name#1206 = finalize)
+- Filter NOT (event_name#1206 = finalize)
 +- Filter NOT (event_name#1206 = finalize)
 +- Filter NOT (event_name#1206 = finalize)
 +- Filter NOT (event_name#1206 = finalize)
 +- Relation[device#1204,ecommerce#1205,event_name#1206,event_previous_timestamp#1207L,event_timestamp#1208L,geo#1209,items#1210,traffic_source#1211,user_first_touch_timestamp#1212L,user_id#1213] parquet

== Optimized Logical Plan ==
Filter (isnotnull(event_name#1206) AND NOT (event_name#1206 = finalize))
+- Relation[device#1204,ecommerce#1205,event_name#1206,event_previous_timestamp#1207L,event_timestamp#1208L,geo#1209,items#1210,traffic_source#1211,user_first_touch_timestamp#1212L,user_id#1213] parquet

== Physical Plan ==
*(1) Filter (isnotnull(event_name#1206) AND NOT (event_name#1206 = finalize))
+- *(1) ColumnarToRow
 +- FileScan parquet [device#1204,ecommerce#1205,event_name#1206,event_previous_timestamp#1207L,event_timestamp#1208L,geo#1209,items#1210,traffic_source#1211,user_first_touch_timestamp#1212L,user_id#1213] Batched: true, DataFilters: [isnotnull(event_name#1206), NOT (event_name#1206 = finalize)], Format: Parquet, Location: InMemoryFileIndex[dbfs:/mnt/training/ecommerce/events/events.parquet], PartitionFilters: [], PushedFilters: [IsNotNull(event_name), Not(EqualTo(event_name,finalize))], ReadSchema: struct<device:string,ecommerce:struct<purchase_revenue_in_usd:double,total_item_quantity:bigint,u...

### ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Predicate Pushdown

Here is example with JDBC where predicate pushdown takes place

In [0]:
%scala
// Ensure that the driver class is loaded
Class.forName("org.postgresql.Driver") 

res1: Class[_] = class org.postgresql.Driver

In [0]:
jdbcURL = "jdbc:postgresql://54.213.33.240/training"

# Username and Password w/read-only rights
connProperties = {
  "user" : "training",
  "password" : "training"
}

ppDF = (spark.read.jdbc(
    url=jdbcURL,                  # the JDBC URL
    table="training.people_1m",   # the name of the table
    column="id",                  # the name of a column of an integral type that will be used for partitioning
    lowerBound=1,                 # the minimum value of columnName used to decide partition stride
    upperBound=1000000,           # the maximum value of columnName used to decide partition stride
    numPartitions=8,              # the number of partitions/connections
    properties=connProperties     # the connection properties
  )
  .filter(col("gender") == "M")   # Filter the data by gender
)

In [0]:
ppDF.explain()

== Physical Plan ==
*(1) Scan JDBCRelation(training.people_1m) [numPartitions=8] [id#1313,firstName#1314,middleName#1315,lastName#1316,gender#1317,birthDate#1318,ssn#1319,salary#1320] PushedFilters: [*IsNotNull(gender), *EqualTo(gender,M)], ReadSchema: struct<id:int,firstName:string,middleName:string,lastName:string,gender:string,birthDate:timestam...

Note the lack of a **Filter** and the presence of a **PushedFilters** in the **Scan**

### ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) No Predicate Pushdown
 
This will make a little more sense if we **compare it to examples** that don't push down the filter.

Caching the data before filtering eliminates the possibility for the predicate push down

In [0]:
cachedDF = (spark.read.jdbc(
    url=jdbcURL,
    table="training.people_1m",
    column="id",
    lowerBound=1,
    upperBound=1000000,
    numPartitions=8,
    properties=connProperties
  ))

cachedDF.cache().count()

filteredDF = cachedDF.filter(col("gender") == "M")

In addition to the **Scan** (the JDBC read) we saw in the previous example, here we also see the **InMemoryTableScan** followed by a **Filter** in the explain plan.

This means Spark had to filter ALL the data from RAM instead of in the Database.

In [0]:
filteredDF.explain()

== Physical Plan ==
*(1) Filter (isnotnull(gender#1640) AND (gender#1640 = M))
+- InMemoryTableScan [id#1636, firstName#1637, middleName#1638, lastName#1639, gender#1640, birthDate#1641, ssn#1642, salary#1643], [isnotnull(gender#1640), (gender#1640 = M)]
 +- InMemoryRelation [id#1636, firstName#1637, middleName#1638, lastName#1639, gender#1640, birthDate#1641, ssn#1642, salary#1643], StorageLevel(disk, memory, deserialized, 1 replicas)
 +- *(1) Scan JDBCRelation(training.people_1m) [numPartitions=8] [id#1636,firstName#1637,middleName#1638,lastName#1639,gender#1640,birthDate#1641,ssn#1642,salary#1643] PushedFilters: [], ReadSchema: struct<id:int,firstName:string,middleName:string,lastName:string,gender:string,birthDate:timestam...

In [0]:
ke

Here is another example using CSV where predicate pushdown does **not** take place

In [0]:
csvDF = (spark.read
  .option("header", "true")
  .option("sep", "\t")
  .option("inferSchema", "true")
  .csv("/mnt/training/wikipedia/pageviews/pageviews_by_second.tsv")
  .filter(col("site") == "desktop"))

Note the presence of a **Filter** and **PushedFilters** in the **FileScan csv**

Again, we see **PushedFilters** because Spark is *trying* to push down to the CSV file.

However, this does not work here, and thus we see, like in the last example, we have a **Filter** after the **FileScan**, actually an **InMemoryFileIndex**.

In [0]:
csvDF.explain()

== Physical Plan ==
*(1) Filter (isnotnull(site#2080) AND (site#2080 = desktop))
+- FileScan csv [timestamp#2079,site#2080,requests#2081] Batched: false, DataFilters: [isnotnull(site#2080), (site#2080 = desktop)], Format: CSV, Location: InMemoryFileIndex[dbfs:/mnt/training/wikipedia/pageviews/pageviews_by_second.tsv], PartitionFilters: [], PushedFilters: [IsNotNull(site), EqualTo(site,desktop)], ReadSchema: struct<timestamp:timestamp,site:string,requests:int>

### Clean up classroom

In [0]:
%run ./Includes/Classroom-Cleanup


Dropped database and removed files in working directory